# Usage with proper cell class documentation

In [1]:
import numpy as np
import pandas as pd
import napari

In [2]:
# add in MGFeatures
def remake_bbox(bbox_asstring, stack=True):
    numbers = ''.join([i if i!='[' and i!=']' and i!='\n ' else '' for i in bbox_asstring])
    arr_flat = np.array([int(i) for i in numbers.split(' ') if i!=''])
    if stack:
        bbox = arr_flat.reshape((4,3))
    else:
        bbox = arr_flat.reshape((4,2))
    return bbox

def make_bbox_stack(slices, bbox_slice):
    sls = [np.array([[sl_num]*4]).T for sl_num in slices]
    bbox_stack = [np.hstack((sl, bbox)) for sl,bbox in zip(sls, bbox_slice)]
    return bbox_stack

## TODO

## Get raw

In [12]:
from Cells import Cells

%load_ext autoreload
%autoreload 2

In [13]:
folder = r'D:\Hanyi temp\MG\ogimg'
raw_stack = []
for i in range(2):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder)
    raw_image = cell.img_from_tiles(folder)
    raw_stack.append(raw_image)

In [7]:
folder = r'D:\Hanyi temp\MG\ogimg'
raw_stack = []
for i in range(17):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder)
    print(imageshape)
    raw_image = cell.img_from_tiles(folder)
    raw_stack.append(raw_image)

(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)
(11, 11, 8192, 8192)


MemoryError: Unable to allocate 7.56 GiB for an array with shape (90112, 90112) and data type uint8

In [40]:
del raw_stack

In [15]:
import sys
print('{} GB'.format(sys.getsizeof(raw_stack)/1024*3))

0.2578125 GB


## Analysis

In [3]:
featuresall = pd.read_excel('featuresall_v05.xlsx')

In [4]:
featuresall.drop(columns='Unnamed: 0', inplace=True)
featuresall

,cell_area,organelles_area,nucleus_area,heterochromatin_area,lipids_area,mitochondria_area,lysosomal_area,empty_area,cytoplasm_area,ER_lengths,...,mito_organelles_R,mito_area,ERlen_cyto_R,hetchr_nucleus_R,Component1,Component2,Component3,Component4,Component5,pca_kmeans_labels
0,80980,28480,39316,17268,3376,604,5896,13184,41664,1576.621487,...,0.021208,604,0.037841,0.439210,1.295136,-1.024184,-0.791488,1.635896,-0.624076,1
1,68545,16588,41028,16854,0,965,0,10929,27517,811.754394,...,0.058175,965,0.029500,0.410793,1.041161,0.148426,0.219833,-0.565449,0.494629,2
2,87034,33185,44767,16235,0,5547,0,9082,42267,1701.741323,...,0.167154,5547,0.040262,0.362656,-1.743781,0.474232,-0.188281,-1.424917,-0.488528,0
3,74590,26943,17393,9114,0,4486,0,30254,57197,1021.955404,...,0.166500,4486,0.017867,0.524004,-2.179071,2.155726,2.508667,3.681094,3.635217,5
4,130431,63809,50680,20881,0,5370,4077,15942,79751,4836.249235,...,0.084157,5370,0.060642,0.412017,-1.009481,-0.610771,-1.025669,0.031985,0.019936,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,162785,73839,59104,26995,8567,9625,0,29842,103681,3982.830848,...,0.130351,9625,0.038414,0.456737,-2.286192,0.254870,0.506727,-0.001963,0.233998,0
930,104354,46189,49812,19212,0,4640,0,8353,54542,2986.833812,...,0.100457,4640,0.054762,0.385690,-1.003962,-0.433802,-0.690478,-1.188833,-0.058032,0
931,257260,171828,76624,31989,109336,4108,0,8808,180636,259.663993,...,0.023908,4108,0.001437,0.417480,0.214660,-2.672200,2.482676,-0.749231,-1.248109,4
932,48279,23758,12364,5365,0,4578,0,12157,35915,964.205186,...,0.192693,4578,0.026847,0.433921,-2.416565,1.375739,1.273675,1.118812,1.448700,5


In [5]:
bbox_rects = [remake_bbox(bbox_str, stack=False) for bbox_str in featuresall['bbox_rects']]

In [6]:
bbox_stack = make_bbox_stack(featuresall['sl_num'], bbox_rects)

In [7]:
pca_kmeans_labels = featuresall['pca_kmeans_labels'].values.tolist()

In [8]:
cmap = {0:'red',
       1:'yellow',
       2:'blue',
       3:'green',
       4:'orange',
       5:'magenta'}
clabels = [cmap.get(k) for k in pca_kmeans_labels]

In [9]:
viewer = napari.Viewer()
viewer.add_shapes(data=bbox_stack,
                 face_color=clabels)

<Shapes layer 'bbox_stack' at 0x24fd2addf90>

In [16]:
viewer.add_image(np.array(raw_stack))

C:\Users\jiangh\.conda\envs\MG\lib\site-packages\napari\_vispy\layers\image.py:231: UserWarning: data shape (90112, 90112) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Image layer 'Image' at 0x24f9a52b670>

C:\Users\jiangh\.conda\envs\MG\lib\site-packages\napari\_vispy\layers\image.py:231: UserWarning: data shape (90112, 90112) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
C:\Users\jiangh\.conda\envs\MG\lib\site-packages\napari\_vispy\layers\image.py:231: UserWarning: data shape (90112, 90112) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
C:\Users\jiangh\.conda\envs\MG\lib\site-packages\napari\_vispy\layers\image.py:231: UserWarning: data shape (90112, 90112) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
C:\Users\jiangh\.conda\envs\MG\lib\site-packages\napari\_vispy\layers\image.py:231: UserWarning: data shape (90112, 90112) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mod

In [38]:
viewer.add_image(np.array(raw_stack))

MemoryError: Unable to allocate 72.2 GiB for an array with shape (17, 139264, 32768) and data type uint8